<center><h1><b>5주차 ✅ 대기오염 데이터 EDA</b></h1></center>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [149]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()

    if system_name == "Darwin":
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux(colab)
        !apt-get install fonts-nanum - qq > /dev/null
        !fc-cache - fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont

        font_family = "NanumBarunGothic"
    return font_family


# style 설정은 꼭 폰트설정 위에서 합니다.
# style 에 폰트 설정이 들어있으면 한글폰트가 초기화 되어 한글이 깨집니다.
plt.style.use("seaborn")
# 폰트설정
plt.rc("font", family=get_font_family())

# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'


----
#### 대기오염 수치

In [41]:
# 시도별 대기오염 월별 수치 데이터 가져오기
air_raw = pd.read_csv('data/pollution/air_1920.csv')
air_raw.head()

,시도명,항목,월,2019,2020
0,강원도,미세먼지 PM10,1,55.769231,32.894737
1,강원도,미세먼지 PM2.5,1,31.923077,22.631579
2,강원도,아황산가스,1,0.003385,0.003000
3,강원도,오존,1,0.022154,0.022316
4,강원도,이산화질소,1,0.018462,0.013684


In [53]:
# 월별마다 수치데이터 그룹화하여 평균값으로 나타내기
air_month_g = air_raw.groupby(by=['시도명', '항목', '월'], as_index=False).mean()
air_month_g.head(11)

,시도명,항목,월,2019,2020
0,강원도,미세먼지 PM10,1,41.634615,30.225146
1,강원도,미세먼지 PM10,2,49.444444,35.157895
2,강원도,미세먼지 PM10,3,57.333333,36.789474
3,강원도,미세먼지 PM10,4,38.055556,39.000000
4,강원도,미세먼지 PM10,5,44.947368,29.222222
5,강원도,미세먼지 PM10,6,28.473684,35.333333
6,강원도,미세먼지 PM10,7,25.368421,16.875000
7,강원도,미세먼지 PM10,8,25.894737,18.263158
8,강원도,미세먼지 PM10,9,20.736842,18.555556
9,강원도,미세먼지 PM10,11,35.235294,32.909091


In [54]:
air_month_g[air_month_g['월'] == 1].head(10)

,시도명,항목,월,2019,2020
0,강원도,미세먼지 PM10,1,41.634615,30.225146
11,강원도,미세먼지 PM2.5,1,22.520362,18.256966
22,강원도,아황산가스,1,0.002803,0.002639
33,강원도,오존,1,0.023105,0.024519
44,강원도,이산화질소,1,0.013592,0.011370
55,강원도,일산화탄소,1,0.504274,0.472222
66,경기도,미세먼지 PM10,1,54.390625,41.953125
77,경기도,미세먼지 PM2.5,1,29.161290,24.609375
88,경기도,아황산가스,1,0.003797,0.003393
99,경기도,오존,1,0.017313,0.019031


In [55]:
air_month_g['항목'].unique()

array(['미세먼지 PM10', '미세먼지 PM2.5', '아황산가스', '오존', '이산화질소', '일산화탄소'],
      dtype=object)

In [84]:
# 등급 나타낼 데이터 프레임 복사하기
air_gd = air_month_g.copy()
air_gd

,시도명,항목,월,2019,2020
0,강원도,미세먼지 PM10,1,41.634615,30.225146
1,강원도,미세먼지 PM10,2,49.444444,35.157895
2,강원도,미세먼지 PM10,3,57.333333,36.789474
3,강원도,미세먼지 PM10,4,38.055556,39.000000
4,강원도,미세먼지 PM10,5,44.947368,29.222222
...,...,...,...,...,...
1117,충청북도,일산화탄소,7,0.325000,0.341667
1118,충청북도,일산화탄소,8,0.358333,0.308333
1119,충청북도,일산화탄소,9,0.316667,0.375000
1120,충청북도,일산화탄소,11,0.525000,0.491667


In [85]:
# 미세먼지 등급 나타내기
def dust_10(x):
    if x <= 30: return '좋음'
    elif x <= 81: return '보통'
    elif x <= 150: return '나쁨'
    else: return '매우나쁨'

def dust_2(x):
    if x <= 15: return '좋음'
    elif x <= 35: return '보통'
    elif x <= 75: return '나쁨'
    else: return '매우나쁨'

In [ ]:
air_gd.loc[air_gd['항목'] == '미세먼지 PM10', '19년_등급'] = \
    air_month_g.loc[air_month_g['항목'] == '미세먼지 PM10', '2019'].map(dust_10)

In [88]:
air_gd.loc[air_gd['항목'] == '미세먼지 PM10', '20년_등급'] = \
    air_month_g.loc[air_month_g['항목'] == '미세먼지 PM10', '2020'].map(dust_10)
air_gd[air_gd['항목'] == '미세먼지 PM10'].head()

,시도명,항목,월,2019,2020,19년_등급,20년_등급
0,강원도,미세먼지 PM10,1,41.634615,30.225146,보통,보통
1,강원도,미세먼지 PM10,2,49.444444,35.157895,보통,보통
2,강원도,미세먼지 PM10,3,57.333333,36.789474,보통,보통
3,강원도,미세먼지 PM10,4,38.055556,39.000000,보통,보통
4,강원도,미세먼지 PM10,5,44.947368,29.222222,보통,좋음


In [ ]:
air_gd.loc[air_gd['항목'] == '미세먼지 PM2.5', '19년_등급'] = \
    air_month_g.loc[air_month_g['항목'] == '미세먼지 PM2.5', '2019'].map(dust_2)

In [91]:
air_gd.loc[air_gd['항목'] == '미세먼지 PM2.5', '20년_등급'] = \
    air_month_g.loc[air_month_g['항목'] == '미세먼지 PM2.5', '2020'].map(dust_2)
air_gd[air_gd['항목'] == '미세먼지 PM2.5'].head()

,시도명,항목,월,2019,2020,19년_등급,20년_등급
11,강원도,미세먼지 PM2.5,1,22.520362,18.256966,보통,보통
12,강원도,미세먼지 PM2.5,2,30.055556,24.578947,보통,보통
13,강원도,미세먼지 PM2.5,3,37.055556,20.842105,나쁨,보통
14,강원도,미세먼지 PM2.5,4,17.277778,18.789474,보통,보통
15,강원도,미세먼지 PM2.5,5,22.473684,14.500000,보통,좋음


In [92]:
# 오존 등급 나타내기
def ozone(x):
    if x <= 0.030: return '좋음'
    elif x <= 0.090: return '보통'
    elif x <= 0.150: return '나쁨'
    else: return '매우나쁨'

In [94]:
air_gd.loc[air_gd['항목'] == '오존', '19년_등급'] = \
    air_month_g.loc[air_month_g['항목'] == '오존', '2019'].map(ozone)

In [95]:
air_gd.loc[air_gd['항목'] == '오존', '20년_등급'] = \
    air_month_g.loc[air_month_g['항목'] == '오존', '2020'].map(ozone)
air_gd[air_gd['항목'] == '오존'].head()

,시도명,항목,월,2019,2020,19년_등급,20년_등급
33,강원도,오존,1,0.023105,0.024519,좋음,좋음
34,강원도,오존,2,0.027778,0.028421,좋음,좋음
35,강원도,오존,3,0.039389,0.035632,보통,보통
36,강원도,오존,4,0.040944,0.043684,보통,보통
37,강원도,오존,5,0.056263,0.039444,보통,보통


In [96]:
# 이산화질소 등급 나타내기
def ntwo(x):
    if x <= 0.02: return '좋음'
    elif x <= 0.05: return '보통'
    elif x <= 0.15: return '나쁨'
    elif x > 0.15: return '매우나쁨'

In [97]:
air_gd.loc[air_gd['항목'] == '이산화질소', '19년_등급'] = \
    air_month_g.loc[air_month_g['항목'] == '이산화질소', '2019'].map(ntwo)


In [98]:
air_gd.loc[air_gd['항목'] == '이산화질소', '20년_등급'] = \
    air_month_g.loc[air_month_g['항목'] == '이산화질소', '2020'].map(ntwo)
air_gd[air_gd['항목'] == '이산화질소'].head()


,시도명,항목,월,2019,2020,19년_등급,20년_등급
44,강원도,이산화질소,1,0.013592,0.011370,좋음,좋음
45,강원도,이산화질소,2,0.014056,0.012158,좋음,좋음
46,강원도,이산화질소,3,0.012667,0.010526,좋음,좋음
47,강원도,이산화질소,4,0.010222,0.008667,좋음,좋음
48,강원도,이산화질소,5,0.009611,0.007000,좋음,좋음


In [99]:
# 일산화탄소 등급 나타내기
def co(x):
    if x <= 2: return '좋음'
    elif x <= 9: return '보통'
    elif x <= 15: return '나쁨'
    else: return '매우나쁨'

In [101]:
air_gd.loc[air_gd['항목'] == '일산화탄소', '19년_등급'] = \
    air_month_g.loc[air_month_g['항목'] == '일산화탄소', '2019'].map(co)


In [102]:
air_gd.loc[air_gd['항목'] == '일산화탄소', '20년_등급'] = \
    air_month_g.loc[air_month_g['항목'] == '일산화탄소', '2020'].map(co)
air_gd[air_gd['항목'] == '일산화탄소'].head()


,시도명,항목,월,2019,2020,19년_등급,20년_등급
55,강원도,일산화탄소,1,0.504274,0.472222,좋음,좋음
56,강원도,일산화탄소,2,0.547059,0.494444,좋음,좋음
57,강원도,일산화탄소,3,0.494444,0.421053,좋음,좋음
58,강원도,일산화탄소,4,0.411111,0.364706,좋음,좋음
59,강원도,일산화탄소,5,0.431579,0.344444,좋음,좋음


In [79]:
# 아황산가스 등급 나타내기
def so(x):
    if x <= 0.03: return '좋음'
    elif x <= 0.06: return '보통'
    elif x <= 0.2: return '나쁨'
    else: return '매우나쁨'

In [103]:
air_gd.loc[air_gd['항목'] == '아황산가스', '19년_등급'] = \
    air_month_g.loc[air_month_g['항목'] == '아황산가스', '2019'].map(so)


In [104]:
air_gd.loc[air_gd['항목'] == '아황산가스', '20년_등급'] = \
    air_month_g.loc[air_month_g['항목'] == '아황산가스', '2020'].map(so)
air_gd[air_gd['항목'] == '아황산가스'].head()


,시도명,항목,월,2019,2020,19년_등급,20년_등급
22,강원도,아황산가스,1,0.002803,0.002639,좋음,좋음
23,강원도,아황산가스,2,0.003056,0.002789,좋음,좋음
24,강원도,아황산가스,3,0.002944,0.002368,좋음,좋음
25,강원도,아황산가스,4,0.002222,0.002526,좋음,좋음
26,강원도,아황산가스,5,0.002421,0.002333,좋음,좋음


In [107]:
air_gd.tail(40)

,시도명,항목,월,2019,2020,19년_등급,20년_등급
1082,충청북도,아황산가스,5,0.003167,0.003083,좋음,좋음
1083,충청북도,아황산가스,6,0.002636,0.002909,좋음,좋음
1084,충청북도,아황산가스,7,0.002583,0.002833,좋음,좋음
1085,충청북도,아황산가스,8,0.002500,0.002750,좋음,좋음
1086,충청북도,아황산가스,9,0.002583,0.002833,좋음,좋음
1087,충청북도,아황산가스,11,0.003083,0.003000,좋음,좋음
1088,충청북도,아황산가스,12,0.003000,0.003333,좋음,좋음
1089,충청북도,오존,1,0.017608,0.020083,좋음,좋음
1090,충청북도,오존,2,0.022167,0.023500,좋음,좋음
1091,충청북도,오존,3,0.034917,0.032583,보통,보통


In [110]:
# 수치형 통계 확인하기
air_gd['월'] = air_gd['월'].astype('object')
air_gd.describe()

,2019,2020
count,1122.000000,1121.000000
mean,10.468310,8.434573
std,16.534303,13.028768
min,0.001667,0.001000
25%,0.014839,0.013000
50%,0.114667,0.052667
75%,20.328125,17.500000
max,71.593750,48.250000


In [111]:
# 범주형 통계 확인하기
air_gd.describe(include='object')

,시도명,항목,월,19년_등급,20년_등급
count,1122,1122,1122,1122,1122
unique,17,6,11,3,3
top,강원도,미세먼지 PM10,1,좋음,좋음
freq,66,187,102,680,744


In [136]:
# 연도별 등급당 최빈값 꺼내기
air_gd_19 = air_gd.groupby(by=['시도명', '항목'])[['19년_등급']].agg(pd.Series.mode)
air_gd_19.head()

19년_등급
시도명 항목               
강원도 미세먼지 PM10      보통
    미세먼지 PM2.5     보통
    아황산가스          좋음
    오존             좋음
    이산화질소          좋음

In [137]:
air_gd_20 = air_gd.groupby(by=['시도명', '항목'])['20년_등급'].agg(pd.Series.mode).to_frame()
air_gd_20.head()

20년_등급
시도명 항목               
강원도 미세먼지 PM10      보통
    미세먼지 PM2.5     보통
    아황산가스          좋음
    오존             좋음
    이산화질소          좋음

In [139]:
air_gd_1920 = pd.merge(air_gd_19, air_gd_20, left_index=True, right_index=True)
air_gd_1920.head()

19년_등급 20년_등급
시도명 항목                      
강원도 미세먼지 PM10      보통     보통
    미세먼지 PM2.5     보통     보통
    아황산가스          좋음     좋음
    오존             좋음     좋음
    이산화질소          좋음     좋음

In [140]:
# 각 대기지수 등급 나타낸 데이터 저장하기
air_gd_1920.to_csv('output/air_grade_1920.csv')

----
#### 대기오염 시각화

In [171]:
# 대기오염 등급 데이터 가져오기
air_gd_raw = pd.read_csv('output/air_grade_1920.csv')
air_gd_raw.shape

(102, 4)

In [172]:
air_gd_raw.head()

,시도명,항목,19년_등급,20년_등급
0,강원도,미세먼지 PM10,보통,보통
1,강원도,미세먼지 PM2.5,보통,보통
2,강원도,아황산가스,좋음,좋음
3,강원도,오존,좋음,좋음
4,강원도,이산화질소,좋음,좋음


In [218]:
# 대기오염 수치 데이터 가져오기
air_raw = pd.read_csv('data/pollution/air_1920.csv')
air_raw.head()

,시도명,항목,월,2019,2020
0,강원도,미세먼지 PM10,1,55.769231,32.894737
1,강원도,미세먼지 PM2.5,1,31.923077,22.631579
2,강원도,아황산가스,1,0.003385,0.003000
3,강원도,오존,1,0.022154,0.022316
4,강원도,이산화질소,1,0.018462,0.013684


In [235]:
air_sido_g = air_raw.groupby(['시도명', '항목'], as_index=False).mean()
air_sido_g = air_sido_g.drop(columns='월')
air_sido_g.head()

,시도명,항목,2019,2020
0,강원도,미세먼지 PM10,37.054471,29.757446
1,강원도,미세먼지 PM2.5,20.787793,16.956441
2,강원도,아황산가스,0.002492,0.002475
3,강원도,오존,0.031794,0.030624
4,강원도,이산화질소,0.010849,0.009395


In [237]:
# 도시별 미세먼지 평균 수치 변화
air_dust = air_sido_g[air_sido_g['항목'] == '미세먼지 PM10']
air_dust.head()

,시도명,항목,2019,2020
0,강원도,미세먼지 PM10,37.054471,29.757446
6,경기도,미세먼지 PM10,45.471102,37.982443
12,경상남도,미세먼지 PM10,37.477595,28.463837
18,경상북도,미세먼지 PM10,39.996434,29.609934
24,광주광역시,미세먼지 PM10,42.000000,29.750000


In [243]:
air_dust = air_dust.melt(id_vars=['시도명', '항목'], value_name='미세먼지 PM10', var_name='시점')
air_dust.head()

,시도명,항목,시점,미세먼지 PM10
0,강원도,미세먼지 PM10,2019,37.054471
1,경기도,미세먼지 PM10,2019,45.471102
2,경상남도,미세먼지 PM10,2019,37.477595
3,경상북도,미세먼지 PM10,2019,39.996434
4,광주광역시,미세먼지 PM10,2019,42.000000


In [295]:
fig = px.histogram(air_dust, x='시도명', y='미세먼지 PM10', color='시점',
        title='연도별 미세먼지 PM10 평균 수치', barmode='group')
fig.show()

In [296]:
# 도시별 미세먼지 평균 수치 변화
air_dust_2 = air_sido_g[air_sido_g['항목'] == '미세먼지 PM2.5']
air_dust_2.head()
air_dust_2 = air_dust_2.melt(
    id_vars=['시도명', '항목'], value_name='미세먼지 PM2.5', var_name='시점')
air_dust_2.head()
fig = px.line(air_dust_2, x='시도명', y='미세먼지 PM2.5', color='시점',
              text='미세먼지 PM2.5',
              markers=True, title='연도별 미세먼지 PM2.5 평균 수치')
fig.update_traces(
    # textposition="auto",
    texttemplate='%{text:.1f}',)
fig.show()


In [302]:
# 도시별 미세먼지 평균 수치 변화
air_no = air_sido_g[air_sido_g['항목'] == '이산화질소']
air_no.head()
air_no = air_no.melt(
    id_vars=['시도명', '항목'], value_name='이산화질소', var_name='시점')
air_no.head()
fig = px.histogram(air_no, x='시도명', y='이산화질소', color='시점',
            title='연도별 이산화질소 평균 수치', barmode='group')
fig.show()


In [303]:
air_sido_g['항목'].unique()

array(['미세먼지 PM10', '미세먼지 PM2.5', '아황산가스', '오존', '이산화질소', '일산화탄소'],
      dtype=object)

In [304]:
# 아황산가스 도시별 수치 나타내기
air_so = air_sido_g[air_sido_g['항목'] == '아황산가스']
air_so.head()
air_so = air_so.melt(
    id_vars=['시도명', '항목'], value_name='아황산가스', var_name='시점')
air_so.head()
fig = px.histogram(air_so, x='시도명', y='아황산가스', color='시점',
            title='연도별 아황산가스 평균 수치', barmode='group')
fig.show()


In [ ]:
# 일산화탄소 도시별 수치 나타내기
air_co = air_sido_g[air_sido_g['항목'] == '일산화탄소']
air_co.head()
air_co = air_co.melt(
    id_vars=['시도명', '항목'], value_name='일산화탄소', var_name='시점')
air_co.head()
fig = px.histogram(air_co, x='시도명', y='일산화탄소', color='시점',
            title='연도별 일산화탄소 평균 수치', barmode='group')
fig.add_vrect(x0='제주특별자치도', x1='제주특별자치도' + 1)
fig.show()

In [305]:
# 오존 도시별 수치 나타내기
air_oz = air_sido_g[air_sido_g['항목'] == '오존']
air_oz.head()
air_oz = air_oz.melt(
    id_vars=['시도명', '항목'], value_name='오존', var_name='시점')
air_oz.head()
fig = px.histogram(air_oz, x='시도명', y='오존', color='시점',
            title='연도별 오존 평균 수치', barmode='group')
fig.show()